In [2]:
import re, ee
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='raman-461708')
PARENT = 'projects/raman-461708/assets'

# ---- List all image assets once (fast follow-up lookups) ----
def _basename(asset_name: str) -> str:
    return asset_name.split('/')[-1]

all_assets = ee.data.listAssets({'parent': PARENT}).get('assets', [])
image_assets = [a for a in all_assets if a.get('type','').lower() == 'image']
basenames = {_basename(a['name']) for a in image_assets}

def asset_exists_base(base: str) -> bool:
    return base in basenames

def fullpath(base: str) -> str:
    return f'{PARENT}/{base}'

def list_part_bases_for_aez(aez_no: int):
    """Return sorted basenames for all parts like AEZ_{n}_v4_local_partK (K numeric)."""
    rx = re.compile(rf'^AEZ_{aez_no}_v4_local_part(\d+)$')
    parts = []
    for b in basenames:
        m = rx.match(b)
        if m:
            parts.append((int(m.group(1)), b))
    # Sort by numeric suffix to keep deterministic order (not required for mosaic, but nice to have).
    parts.sort(key=lambda x: x[0])
    return [b for _, b in parts]

def choose_assets_for_aez(aez_no: int):
    """
    Preference:
      1) AEZ_{n}_v4
      2) AEZ_{n}_v4_local_part*
      3) AEZ_{n}_v4_local
    Returns: list of full asset IDs (possibly multiple if parts), plus a metadata dict.
    """
    base_v4 = f'AEZ_{aez_no}_v4'
    base_local = f'AEZ_{aez_no}_v4_local'
    part_bases = list_part_bases_for_aez(aez_no)

    if asset_exists_base(base_v4):
        return [fullpath(base_v4)], {'source':'v4','parts':[]}

    if part_bases:
        return [fullpath(b) for b in part_bases], {'source':'v4_local_parts','parts':part_bases}

    if asset_exists_base(base_local):
        return [fullpath(base_local)], {'source':'v4_local','parts':[]}

    return [], {'source':'missing','parts':[]}

# ---- Collect images AEZ 1..19 ----
images = []
report = {}   # aez_no -> {'source':..., 'parts':[...] }
missing = []

for aez_no in range(1, 20):
    asset_ids, meta = choose_assets_for_aez(aez_no)
    report[aez_no] = meta
    if asset_ids:
        # If this AEZ has multiple parts, add them all; mosaic will merge them.
        for aid in asset_ids:
            images.append(ee.Image(aid))
    else:
        missing.append(aez_no)

if not images:
    raise RuntimeError("No AEZ images found across v4 / v4_local_parts / v4_local")

roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions")

# ---- Final mosaic across all gathered images ----
# Order: AEZ 1..19; within an AEZ, parts in ascending numeric order.
# ImageCollection.mosaic() draws later images on top of earlier ones.
lulc_v4 = ee.ImageCollection(images).mosaic()
lulc_v3 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024').select('predicted_label')
farm_boundaries = ee.FeatureCollection('projects/raman-461708/assets/india_10k_fields')

dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2024-07-01', '2025-07-01').filterBounds(farm_boundaries)
dw = dw.select('label').mode().clip(farm_boundaries)


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [2]:
import geemap
m = geemap.Map()
#m.centerObject(farm.limit(1))


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 0,
    'max': 2,
    'palette': palette_
}
m.addLayer(lulc_v4.clip(farm_boundaries).select("predicted_label"), vis_params_corrected, 'lulc_v4')
m.addLayer(lulc_v3.clip(farm_boundaries).select("predicted_label"), vis_params, 'lulc_v3')
m.addLayer(farm_boundaries, {}, 'Field Boundaries')
m.centerObject(farm_boundaries.limit(1),17)
m

Map(center=[26.302457463638135, 73.10706082645409], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
farm_boundaries = ee.FeatureCollection('projects/raman-461708/assets/india_10k_fields')

lulc = lulc_v3
# Inputs:
# lulc: ee.Image with band 'predicted_label' (already clipped to farm_boundaries)
# farm_boundaries: ee.FeatureCollection
band    = 'predicted_label'
classes = [0,1,2,3,4,5,6,7,8, 9, 10, 11, 12, 13]

mask = lulc.select(band).remap(classes, [1]*len(classes), 0)
area_img = ee.Image.pixelArea().rename('area_m2').updateMask(mask)

def get_area(feat):
    geom = feat.geometry()
    # Total area in m² and hectares
    stats   = area_img.reduceRegion(
        reducer   = ee.Reducer.sum(),
        geometry  = geom,
        scale     = 10,
        maxPixels = 1e13,
        tileScale = 4
    )
    area_m2 = ee.Number(stats.get('area_m2'))
    return area_m2

farm_boundaries = farm_boundaries.map(lambda f: f.set('area_m2', get_area(f)))
area_inside = farm_boundaries.aggregate_array('area_m2').getInfo()
print(sum(area_inside))

42895801.75259463


In [57]:
farm_boundaries.geometry().area().getInfo()

43028355.14236441

In [3]:
34736957.10651489/42899749.586338215*100

80.97240063512439

In [4]:
34740338.97414855/42899749.586338215*100

80.9802838224769

In [5]:
35639174.87927785/42899749.586338215*100

83.07548464251978

In [5]:
farm_boundaries = ee.FeatureCollection('projects/raman-461708/assets/india_10k_fields')
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2024-07-01', '2025-07-01').filterBounds(farm_boundaries)
lulc = dw.select('label').mode().clip(farm_boundaries)
task = ee.batch.Export.image.toAsset(
        image=lulc,#.select("predicted_label"),
        description='lulc_dw_10k_fields_2024_2025',
        assetId='projects/raman-461708/assets/dw_10k_fields_2024_2025',
        pyramidingPolicy = {'predicted_label': 'mode'},
        scale = 10,
        maxPixels = 1e13,
        crs = 'EPSG:4326'
    )
task.start()

In [3]:
farm_boundaries = ee.FeatureCollection('projects/raman-461708/assets/india_10k_fields')
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2024-07-01', '2025-07-01').filterBounds(farm_boundaries)
lulc = dw.select('label').mode().clip(farm_boundaries)

# Inputs:
# lulc: ee.Image with band 'predicted_label' (already clipped to farm_boundaries)
# farm_boundaries: ee.FeatureCollection
band    = 'label'
classes = [4]

mask = lulc.select(band).remap(classes, [1]*len(classes), 0)
area_img = ee.Image.pixelArea().rename('area_m2').updateMask(mask)

def get_area(feat):
    geom = feat.geometry()
    # Total area in m² and hectares
    stats   = area_img.reduceRegion(
        reducer   = ee.Reducer.sum(),
        geometry  = geom,
        scale     = 10,
        maxPixels = 1e13,
        tileScale = 4
    )
    area_m2 = ee.Number(stats.get('area_m2'))
    return area_m2

farm_boundaries = farm_boundaries.limit(1000).map(lambda f: f.set('area_m2', get_area(f)))
area_inside = farm_boundaries.aggregate_array('area_m2').getInfo()
print(sum(area_inside))

EEException: Computation timed out.

In [ ]:
farm_boundaries = ee.FeatureCollection('projects/raman-461708/assets/india_10k_fields')
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2024-07-01', '2025-07-01').filterBounds(farm_boundaries)
lulc = dw.select('label').mode().clip(farm_boundaries)

# Inputs:
# lulc: ee.Image with band 'predicted_label' (already clipped to farm_boundaries)
# farm_boundaries: ee.FeatureCollection
band    = 'label'
classes = [4]

mask = lulc.select(band).remap(classes, [1]*len(classes), 0)
area_img = ee.Image.pixelArea().rename('area_m2').updateMask(mask)

def get_area(feat):
    geom = feat.geometry()
    # Total area in m² and hectares
    stats   = area_img.reduceRegion(
        reducer   = ee.Reducer.sum(),
        geometry  = geom,
        scale     = 10,
        maxPixels = 1e13,
        tileScale = 4
    )
    area_m2 = ee.Number(stats.get('area_m2'))
    return area_m2

farm_boundaries = farm_boundaries.map(lambda f: f.set('area_m2', get_area(f)))
area_inside = farm_boundaries.aggregate_array('area_m2').getInfo()
#print(sum(area_inside))

task = ee.batch.Export.image.toAsset(
    image=lulc.select('label'),#.select("predicted_label"),
    description='lulc_dw',
    assetId='projects/ee-raman/assets/dw_10000_fields_2024',
    pyramidingPolicy = {'predicted_label': 'mode'},
    scale = 10,
    maxPixels = 1e13,
    crs = 'EPSG:4326'
)
task.start()

KeyboardInterrupt: 

In [2]:
farm_boundaries = ee.FeatureCollection('projects/raman-461708/assets/india_10k_fields')
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2024-07-01', '2025-07-01').filterBounds(farm_boundaries)
lulc = dw.select('label').mode().clip(farm_boundaries)
task = ee.batch.Export.image.toAsset(
    image=lulc.select('label'),#.select("predicted_label"),
    description='lulc_dw',
    assetId='projects/raman-461708/assets/dw_10000_fields_2024',
    pyramidingPolicy = {'predicted_label': 'mode'},
    scale = 10,
    maxPixels = 1e13,
    crs = 'EPSG:4326'
)
task.start()

In [6]:
roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions")
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2024-07-01', '2025-07-01').filterBounds(roi)
lulc = dw.select('label').mode().clip(roi)
task = ee.batch.Export.image.toAsset(
    image=lulc.select('label'),#.select("predicted_label"),
    description='lulc_dw',
    assetId='projects/raman-461708/assets/dw_10000_fields_2024_india',
    pyramidingPolicy = {'predicted_label': 'mode'},
    scale = 10,
    maxPixels = 1e13,
    crs = 'EPSG:4326'
)
task.start()

In [5]:
roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions")
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2024-07-01', '2025-07-01')
for i in range(20):
    aez_no = i
    roi_aez = roi.filter(ee.Filter.eq("ae_regcode", aez_no)) 
    lulc = dw.filterBounds(roi_aez).select('label').mode().clip(roi_aez)
    task = ee.batch.Export.image.toAsset(
        image=lulc.select('label'),#.select("predicted_label"),
        description='lulc_dw_' + str(aez_no),
        assetId='projects/raman-461708/assets/dw_10000_fields_2024_' + str(aez_no),
        pyramidingPolicy = {'predicted_label': 'mode'},
        scale = 10,
        maxPixels = 1e13,
        crs = 'EPSG:4326'
    )
    task.start()